In [ ]:
import pandas as pd
from pathlib import Path
from collections import deque
from tqdm import tqdm
import os
import csv

from pipeline.aggregator import TimeframeAggregator
from data_gen.generate_plain_img import ImageGenerator  # plain candles + YOLO boxes

# --- Config ---
csv_path = "./data/agg_data/fx/C:EURUSD_1m_last1y.csv"
save_dir = Path("./dataset")

timeframes = ["5m", "15m", "1h", "4h", "1d"]
candle_limits = {tf: 60 for tf in timeframes}  # always 60 candles per TF
image_size = (640, 640)

# --- Setup ---
df = pd.read_csv(csv_path, parse_dates=["timestamp"])
buffer = deque(maxlen=60 * 24 * 60 + 100)
aggregator = TimeframeAggregator(buffer)
image_gen = ImageGenerator(candle_limits, image_size=image_size)

# --- Warmup with first 60 days of 1m data ---
start_index = 60 * 24 * 60  # 60 days * 24h * 60m
print(f"[INFO] Warming up with first {start_index} minutes ({60} days)")

warmup_df = df.iloc[:start_index]
for _, row in tqdm(warmup_df.iterrows(), total=len(warmup_df), desc="Warming buffer"):
    buffer.append(row.to_dict())
    aggregator.resample_all(timeframes)

print("[INFO] Warmup complete. Starting main generation loop.")

# --- Prepare meta.csv ---
meta_path = save_dir / "meta.csv"
os.makedirs(save_dir, exist_ok=True)

header = ["timestamp", "close"]
for tf in timeframes:
    header.append(f"{tf}_img")
    header.append(f"{tf}_lbl")

with open(meta_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()

# --- Main loop (forward from day 60) ---
for i, row in tqdm(df.iloc[start_index:].iterrows(),
                   total=len(df) - start_index,
                   desc="Generating images (every 5m, 60-bar windows per TF)"):
    bar = row.to_dict()
    buffer.append(bar)

    resampled = aggregator.resample_all(timeframes)
    timestamp = pd.to_datetime(row["timestamp"])
    close_price = row["close"]

    # Only produce outputs on 5m boundaries
    if timestamp != timestamp.floor("5min"):
        continue

    tf_image_paths = {}
    for tf in timeframes:
        df_tf = resampled[tf]
        if df_tf.empty:
            continue

        # Always grab up to 60 candles (partial if fewer)
        window = df_tf.tail(min(len(df_tf), candle_limits[tf]))

        img_path = save_dir / "images" / tf / f"{timestamp}.png"
        lbl_path = save_dir / "labels" / tf / f"{timestamp}.txt"

        img_path.parent.mkdir(parents=True, exist_ok=True)
        lbl_path.parent.mkdir(parents=True, exist_ok=True)

        image_gen.generate_image(tf, window, str(img_path))

        tf_image_paths[f"{tf}_img"] = str(img_path)
        tf_image_paths[f"{tf}_lbl"] = str(lbl_path)

    # Write CSV row even if some TFs are missing (they’ll just be blank)
    row_out = {"timestamp": timestamp, "close": close_price}
    row_out.update(tf_image_paths)

    with open(meta_path, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writerow(row_out)

print(f"[✅ DONE] Meta written (every 5m, all TFs, partial candles allowed) to {meta_path}")




In [ ]:
import os
import glob
import shutil

# --- Settings ---
SOURCE_IMAGE_DIR = "./dataset/images"
SOURCE_LABEL_DIR = "./dataset/labels"
OUTPUT_BASE_DIR = "./img_dataset"

TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.2
TEST_SPLIT = 0.1
MAX_IMAGES = 200_000

# --- Collect all image paths (sorted order) ---
image_paths = sorted(
    glob.glob(f"{SOURCE_IMAGE_DIR}/**/*.png", recursive=True) +
    glob.glob(f"{SOURCE_IMAGE_DIR}/**/*.jpg", recursive=True)
)

# --- Limit to MAX_IMAGES ---
total = min(len(image_paths), MAX_IMAGES)
image_paths = image_paths[:total]

# --- Calculate split indices ---
n_train = int(total * TRAIN_SPLIT)
n_val = int(total * VAL_SPLIT)
n_test = total - n_train - n_val

train_imgs = image_paths[:n_train]
val_imgs = image_paths[n_train:n_train + n_val]
test_imgs = image_paths[n_train + n_val:]

splits = {
    "train": train_imgs,
    "val": val_imgs,
    "test": test_imgs
}

# --- Copy files with timeframe in name ---
for split, paths in splits.items():
    for img_path in paths:
        relative_img_path = os.path.relpath(img_path, SOURCE_IMAGE_DIR)
        tf = relative_img_path.split(os.sep)[0]  # timeframe (e.g. "1h")

        filename = os.path.basename(img_path)
        new_filename = f"{tf}_{filename}"

        img_out_dir = os.path.join(OUTPUT_BASE_DIR, "images", split)
        lbl_out_dir = os.path.join(OUTPUT_BASE_DIR, "labels", split)
        os.makedirs(img_out_dir, exist_ok=True)
        os.makedirs(lbl_out_dir, exist_ok=True)

        # Copy image
        shutil.copy(img_path, os.path.join(img_out_dir, new_filename))

        # Copy label if exists
        label_rel_path = os.path.splitext(relative_img_path)[0] + ".txt"
        label_full_path = os.path.join(SOURCE_LABEL_DIR, label_rel_path)

        if os.path.exists(label_full_path):
            new_label_name = os.path.splitext(new_filename)[0] + ".txt"
            shutil.copy(label_full_path, os.path.join(lbl_out_dir, new_label_name))
        else:
            print(f"⚠️ Label not found for: {img_path}")

print("✅ Dataset split complete. Filenames now include timeframe.")
print(f"Total: {total} | Train: {len(train_imgs)} | Val: {len(val_imgs)} | Test: {len(test_imgs)}")



In [ ]:
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

from env.trading_env import TradingEnv
from models.yolo_extractor import CustomYOLOPolicy

from torch.utils.tensorboard import SummaryWriter

# writer = SummaryWriter(log_dir="./logs/custom")



# --- Load your meta.csv ---
meta_df = pd.read_csv("./dataset/meta.csv", parse_dates=["timestamp"])

# --- Create your environment ---
def make_env():
    def _init():
        env = TradingEnv(meta_df)
        env = Monitor(env)  # Wrap with Monitor to log rewards
        return env
    return _init

# Wrap in vectorized and monitored env
vec_env = DummyVecEnv([make_env()])
vec_env = VecMonitor(vec_env)  # Logs mean reward per episode

from stable_baselines3.common.vec_env import VecNormalize

vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True, clip_reward=10.0)

# --- Create PPO model ---
model = PPO(
    policy=CustomYOLOPolicy,
    env=vec_env,
    verbose=1,
    n_steps=128,
    batch_size=32,
    learning_rate=3e-4,
    ent_coef=0.01,
    tensorboard_log="./logs"
)

# After defining your model:
# callback = RewardLoggingCallback(writer=writer)


# --- Train the model ---

model.learn(total_timesteps=100000)

vec_env.save("vec_normalize.pkl")

# --- Save it ---
model.save("ppo-yolo-trading")
print("✅ Training complete. Model saved to: ppo-yolo-trading")

